---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [1]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import pandas as pd
    import numpy as np
    Cityu = pd.read_csv('university_towns.txt', sep=";", names=['RegionName'])
    Cityu.insert(0, 'State', Cityu['RegionName'].str.extract('(.*)\[edit\]', expand=False).ffill())
    Cityu['RegionName'] = Cityu['RegionName'].str.replace(r' \(.+$', '')
    Cityu = Cityu[~Cityu['RegionName'].str.contains('\[edit\]')].reset_index(drop=True)
    return Cityu
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [2]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    file = 'gdplev.xls'
    df = pd.read_excel(file,skiprows = 5, na_values = ['...']).drop(['Unnamed: 3','Unnamed: 7'],axis=1)
    df = df.rename(columns ={'GDP in billions of current dollars':'GDP','GDP in billions of chained 2009 dollars':'GDP_2009','Unnamed: 0':'Year','Unnamed: 4':'Qs','GDP in billions of current dollars.1':'GDP1','GDP in billions of chained 2009 dollars.1':'GDP1_2009'})
    df['Recesion'] = np.where((df['GDP1'].shift(4) > df['GDP1'].shift(3)) & (df['GDP1'].shift(3) > df['GDP1'].shift(2)) & (df['GDP1'].shift(2) < df['GDP1'].shift(1)) & (df['GDP1'].shift(1) < df['GDP1'].shift(0)),df['Qs'].shift(5),"No")
    return df['Recesion'][df['Recesion'] != 'No'].values[2]
get_recession_start()

'2008q3'

In [17]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    file = 'gdplev.xls'
    df = pd.read_excel(file,skiprows = 5, na_values = ['...']).drop(['Unnamed: 3','Unnamed: 7'],axis=1)
    df = df.rename(columns ={'GDP in billions of current dollars':'GDP','GDP in billions of chained 2009 dollars':'GDP_2009','Unnamed: 0':'Year','Unnamed: 4':'Qs','GDP in billions of current dollars.1':'GDP1','GDP in billions of chained 2009 dollars.1':'GDP1_2009'})
    df['Recesion'] = np.where((df['GDP1'].shift(4) > df['GDP1'].shift(3)) & (df['GDP1'].shift(3) > df['GDP1'].shift(2)) & (df['GDP1'].shift(2) < df['GDP1'].shift(1)) & (df['GDP1'].shift(1) < df['GDP1'].shift(0)),df['Qs'],"No")
    return df['Recesion'][df['Recesion'] != 'No'].values[2]
get_recession_end()

'2009q4'

In [3]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    import numpy as np
    file = 'gdplev.xls'
    df = pd.read_excel(file,skiprows = 5, na_values = ['...']).drop(['Unnamed: 3','Unnamed: 7'],axis=1)
    df = df.rename(columns ={'GDP in billions of current dollars':'GDP','GDP in billions of chained 2009 dollars':'GDP_2009','Unnamed: 0':'Year','Unnamed: 4':'Qs','GDP in billions of current dollars.1':'GDP1','GDP in billions of chained 2009 dollars.1':'GDP1_2009'})
    df['Recesion'] = np.where((df['GDP1'].shift(4) > df['GDP1'].shift(3)) & (df['GDP1'].shift(3) > df['GDP1'].shift(2)) & (df['GDP1'].shift(2) < df['GDP1'].shift(1)) & (df['GDP1'].shift(1) < df['GDP1'].shift(0)),df['Qs'].shift(2),"No")
    return df['Recesion'][df['Recesion'] != 'No'].values[2]
get_recession_bottom()

'2009q2'

In [1]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    file = 'City_Zhvi_AllHomes.csv'
    df   = pd.read_csv(file,header=0)
    df['State'].replace(states, inplace= True)
    df   = df.set_index(['State','RegionName'])
    df   = df.drop(df.iloc[:,0:49], axis=1)
    df   = df.groupby(pd.PeriodIndex(df.columns, freq = 'q'),axis=1).mean()
    df   = df.sort_index()
    return df
convert_housing_data_to_quarters()

FileNotFoundError: [Errno 2] File City_Zhvi_AllHomes.csv does not exist: 'City_Zhvi_AllHomes.csv'

In [5]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    import pandas as pd
    import numpy  as np
    df1 = get_list_of_university_towns()
    df4 = convert_housing_data_to_quarters() 
    df4['Ratio'] = df4['2008Q3'] - df4['2009Q2']
    dfs = pd.merge(df4, df1, on=['State', 'RegionName'])
    dfs['Town_Values'] = 1
    df4 = pd.merge(df4, dfs, how='outer', on=['State','RegionName','Ratio'])
    df4['Town_Values'] = df4['Town_Values'].fillna(0)
    Hucieties =  df4[df4['Town_Values'] == 1]
    Nonucities = df4[df4['Town_Values']== 0]
    from scipy import stats
    t,p   =  stats.ttest_ind(Hucieties['Ratio'].dropna(), Nonucities['Ratio'].dropna())
    different = True if p < 0.01 else False
    better    = 'university town' if Hucieties['Ratio'].mean() < Nonucities['Ratio'].mean() else 'non-university'
    return different,p,better
run_ttest()

(True, 0.004325214853511201, 'university town')